In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

In [4]:
# Automobile(1) = Class 0, Horse(7) = Class 1
x17_train = x_train[(y_train == 1) | (y_train == 7),]
y17_train = y_train[(y_train == 1) | (y_train == 7)]
y17_train = np.where(y17_train == 1, 0, 1).reshape((-1,1))

x17_test = x_test[(y_test == 1) | (y_test == 7),]
y17_test = y_test[(y_test == 1) | (y_test == 7)]
y17_test = np.where(y17_test == 1, 0, 1).reshape((-1,1))

In [5]:
x17_train = x17_train/255
x17_test = x17_test/255

In [6]:
base_model = VGG16(input_shape = (x17_train[0].shape[0],x17_train[0].shape[1],3),
                   include_top = False, weights = 'imagenet')

In [7]:
model_name = 'cifar10horseautomobile_oracle'

inputs = tf.keras.Input(shape=(x17_train[0].shape[0],x17_train[0].shape[1],3))
x = base_model(inputs, training=False)
x = Flatten()(x)
outputs = Dense(1, activation = 'sigmoid')(x)
model = tf.keras.Model(inputs, outputs, name = model_name)
model.summary()

Model: "cifar10horseautomobile_oracle"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 14,715,201
Non-trainable params: 0
_________________________________________________________________


In [8]:
train_gen = ImageDataGenerator(
    rotation_range=20,
    shear_range=10, validation_split = 0.2)
train_gen.fit(x17_train)

In [9]:
lr = 0.0001
batchsize = 50

path ="./oracles/cifar10horseautomobile.h5"

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_accuracy', patience = 3, restore_best_weights=True)
checkpoint = ModelCheckpoint(path, monitor = 'val_accuracy', save_best_only=True)
callbacks = [earlystop]

In [10]:
model.fit(train_gen.flow(x17_train, y17_train, batch_size = batchsize, subset = 'training'),
          validation_data=train_gen.flow(x17_train, y17_train, batch_size = batchsize, subset = 'validation'),
          steps_per_epoch=len(x17_train)*0.8/batchsize,
          epochs = 100, 
          callbacks = callbacks)
model.save(path, save_format = 'h5')

Epoch 1/100
160/160 [==============================] - 15s 41ms/step - loss: 0.1579 - accuracy: 0.9310 - val_loss: 0.0559 - val_accuracy: 0.9805
Epoch 2/100
160/160 [==============================] - 6s 40ms/step - loss: 0.0540 - accuracy: 0.9806 - val_loss: 0.0530 - val_accuracy: 0.9820
Epoch 3/100
160/160 [==============================] - 6s 40ms/step - loss: 0.0381 - accuracy: 0.9866 - val_loss: 0.0344 - val_accuracy: 0.9885
Epoch 4/100
160/160 [==============================] - 6s 39ms/step - loss: 0.0316 - accuracy: 0.9893 - val_loss: 0.0913 - val_accuracy: 0.9645
Epoch 5/100
160/160 [==============================] - 7s 41ms/step - loss: 0.0234 - accuracy: 0.9916 - val_loss: 0.0241 - val_accuracy: 0.9895
Epoch 6/100
160/160 [==============================] - 6s 39ms/step - loss: 0.0148 - accuracy: 0.9951 - val_loss: 0.0989 - val_accuracy: 0.9680
Epoch 7/100
160/160 [==============================] - 6s 40ms/step - loss: 0.0161 - accuracy: 0.9939 - val_loss: 0.0309 - val_accuracy

In [11]:
model = load_model(path)
model.summary()

Model: "cifar10horseautomobile_oracle"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 14,715,201
Non-trainable params: 0
_________________________________________________________________


In [12]:
preds = model.predict(x17_test)
preds = np.squeeze(np.round(preds, 0).astype('int'))

In [13]:
np.mean(preds == np.squeeze(y17_test))

0.993